In [1]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import time
import json


In [2]:
load_dotenv()

True

In [3]:
test_address = 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU'
test_wallet_timeseries = pd.read_csv('../data/processed/test_acc_timeseries.csv')

test_wallet_timeseries['timestamp'] = pd.to_datetime(test_wallet_timeseries['timestamp'])
test_wallet_timeseries = test_wallet_timeseries.sort_values(by='timestamp',ascending=False)
test_wallet_timeseries['sender'].head()
unique_receivers = test_wallet_timeseries['receiver'].dropna().unique()
unique_senders = test_wallet_timeseries['sender'].dropna().unique()
unique_addresses = list(set(unique_receivers.tolist() + unique_senders.tolist()))
len(unique_addresses)

241

# Flipside Labels

https://flipsidecrypto.xyz/studio/queries/d238f65d-0fdf-4f56-b7df-6ec6d95a0559

In [4]:
flipside_files = ['solana_cex_labels','solana_chadmin','solana_dapp_labels','solana_defi_labels']
flipside_labels_df = pd.DataFrame()
for file in flipside_files:
    dir = f'{file}.csv'
    df = pd.read_csv(f"../data/raw/{file}.csv", on_bad_lines='skip').dropna()
    flipside_labels_df = pd.concat([flipside_labels_df,df])


In [5]:
flipside_labels_df = flipside_labels_df.drop_duplicates(subset='ADDRESS')
flipside_labels_df

,ADDRESS,LABEL_TYPE,LABEL_SUBTYPE,ADDRESS_NAME,PROJECT_NAME
0,F7nqmScTgLEGqB13dg2VQeURpk7htKvkdcxK5fXZGoqk,cex,deposit_wallet,kucoin deposit_wallet,kucoin
1,DN8hF8iv1jmktXdYAfs2mAkoLycpYimU7xE5LWA8umrr,cex,deposit_wallet,coinbase deposit_wallet,coinbase
2,Dw9wBygiwVWF9DHQFoEYPb5AqJmBBEcdiJnkhDYajFTd,cex,deposit_wallet,ftx deposit_wallet,ftx
3,DZJEkYa9zoMYrMkwhvh3Qen5ER8Ga6voRDJDCZBR7Ykz,cex,deposit_wallet,binance deposit_wallet,binance
4,58z1n26qxUdHukASYvsk7VoHMNH4LDxWMvouXVVi5rXj,cex,deposit_wallet,backpack exchange deposit_wallet,backpack exchange
...,...,...,...,...,...
1474,H8h7ZyS5qJR2cwLxvZQdPaNzLik17cRxB5pDvjdXeuBg,defi,router,kamino: sol/wif strategy address,kamino
1475,12kG3nQHrp9hozgBS2PYAKdo5SMtJRELfKYjjwvwrbhX,defi,pool,frakt: taiyo pilots pool,frakt
1476,8KvNpeyKyZYWfHsBtds4fRLHXRaMU9aop43eagXffdU,defi,general_contract,upfi,upfi
1477,2hdg3iYYz4ZkyhUVQU4uzQybiCEfkQ6kYvMuGmtXUYg2,defi,pool,frakt: communi3: mad scientists pool,frakt


In [6]:
flipside_labels_dict = dict(zip(flipside_labels_df['ADDRESS'], flipside_labels_df['ADDRESS_NAME']))
flipside_labels_dict

{'F7nqmScTgLEGqB13dg2VQeURpk7htKvkdcxK5fXZGoqk': 'kucoin deposit_wallet',
 'DN8hF8iv1jmktXdYAfs2mAkoLycpYimU7xE5LWA8umrr': 'coinbase deposit_wallet',
 'Dw9wBygiwVWF9DHQFoEYPb5AqJmBBEcdiJnkhDYajFTd': 'ftx deposit_wallet',
 'DZJEkYa9zoMYrMkwhvh3Qen5ER8Ga6voRDJDCZBR7Ykz': 'binance deposit_wallet',
 '58z1n26qxUdHukASYvsk7VoHMNH4LDxWMvouXVVi5rXj': 'backpack exchange deposit_wallet',
 '94Pzars8ccjgiGgFJ3y6SGXBx7FC3JjBSDL34sArVJZk': 'kraken deposit_wallet',
 'FSUsvp5wk9BwHHQMLcWVHgb1hKFuXSb5RG6suuKi1294': 'backpack exchange deposit_wallet',
 'fznuAN32RGr2CpYUhSqHBb9v3LaiBHJFDSksZGV4NzL': 'coinbase deposit_wallet',
 'Au7gWXZSwQdULT6NN82JaE8FUeNKrvaZAZakWAcVbUoX': 'kucoin deposit_wallet',
 'BBiAMmmo9QtsW6gfzPjxEngNxkbf9RDNNQa92egk1Q3J': 'backpack exchange deposit_wallet',
 'FkDzh9GMam7bLPDqsh3kdcwvvFGCdfKqsSqumTNxfXLH': 'bybit deposit_wallet',
 '6TbjfKvdhihs192ZTHXBpRgoHyzgNdy18K13aEJUpgs3': 'gate.io deposit_wallet',
 '6ddL3YL9GEB1AFa5cyMs7ga2WC33CATpVrTcysDKzPNb': 'mexc deposit_wallet',
 '29ev

# Vybe

In [7]:
VYBE_API_KEY = os.getenv('VYBE_API_KEY')

In [8]:
# Ideally we can cache this weekly; likely doesnt change too frequently

def get_vybe_identified_accounts(use_cache=True):

    url = "https://api.vybenetwork.xyz/account/known-accounts"

    headers = {"accept": "application/json","X-API-KEY":VYBE_API_KEY}

    response = requests.get(url, headers=headers)

    data = response.json()

    identified_addresses = {}
    for account in data['accounts']:
        address = account.get('ownerAddress')
        address = address.lower()
        name = account.get('name')
        identified_addresses[address] = name

    return identified_addresses

In [9]:
# Ideally we can cache this weekly; likely doesnt change too frequently

def get_vybe_identified_programs(use_cache=True):

    url = "https://api.vybenetwork.xyz/program/known-program-accounts"

    headers = {"accept": "application/json","X-API-KEY":VYBE_API_KEY}

    response = requests.get(url, headers=headers)

    data = response.json()

    identified_programs = {}

    for account in data['programs']:
        program_id = account.get('programId')
        program_id = program_id.lower()
        name = account.get('name')
        identified_programs[program_id] = name

    return identified_programs

In [10]:
vybe_programs_map = get_vybe_identified_programs()
vybe_addresses_map = get_vybe_identified_accounts()

In [11]:
# # Normalize mapping keys
# vybe_programs_map = {k.lower(): v for k, v in vybe_programs_map.items()}
# vybe_addresses_map = {k.lower(): v for k, v in vybe_addresses_map.items()}

# # Apply human-readable labels to tx-level DataFrame
# test_wallet_timeseries['sender_name'] = test_wallet_timeseries['sender'].astype(str).str.lower().map(
#     lambda addr: vybe_addresses_map.get(addr, 'Unknown Address'))

# test_wallet_timeseries['receiver_name'] = test_wallet_timeseries['receiver'].astype(str).str.lower().map(
#     lambda addr: vybe_addresses_map.get(addr, 'Unknown Address'))

# test_wallet_timeseries['counterparty_name'] = test_wallet_timeseries['counterparty'].astype(str).str.lower().map(
#     lambda addr: vybe_addresses_map.get(addr, 'Unknown Address'))

# test_wallet_timeseries['program_name'] = test_wallet_timeseries['program_id'].astype(str).str.lower().map(
#     lambda pid: vybe_programs_map.get(pid, 'Unknown Program'))

# test_wallet_timeseries['wallet_entity_label'] = vybe_addresses_map.get(test_address.lower(), 'Unknown Entity')

In [12]:
# 'sender_name', 'receiver_name', 'counterparty_name', 'program_name',
#        'wallet_entity_label'

In [13]:
list(vybe_programs_map.values())

['Star Atlas SAGE',
 'Cardinal Token Manager',
 'Pyth Staking',
 'Star Atlas - PROFILE ACTION',
 'Hadeswap',
 'Helium Voter Stake Registry',
 'HXRO - Risk',
 'Citrus',
 'MPL Auction House',
 'Zeta Markets',
 'Star Atlas - CRAFTING',
 'Marginfi',
 'Orca-Openbook Bot',
 'Stabble Stable Swap',
 'Monaco Protocol',
 'Star Atlas Proxy Rewarder',
 'Raydium AMM Route',
 'Cykura',
 'Titan Dex ',
 'Streamflow MultiSig',
 'Parrot Protocol',
 'Socean Program',
 'Dradex',
 'Solend',
 'GooseFx v1',
 'Jet Protocol Rewards',
 'Zeta Matching Engine',
 'Crema Finance',
 'Tensor Compressed',
 'Taiyo Launchpad',
 'MPL Candy Guard',
 'Jupiter Swap V3',
 'Meta DAO Futarchy',
 'GooseFX Controller',
 'Marinade - Directed Stake',
 'Hub3.ee',
 'Stabble Weighted Swap',
 'Star Atlas FACTION ENLISTMENT',
 'Raydium CLMM Program',
 'Aldrin AMM V1',
 'Fluxbeam',
 'Jupiter v5',
 "HXRO - Fee's",
 ' Squads Program ID V4',
 'Meteora DLMM Program',
 'MPL Bubblegum',
 'Streamflow',
 'Orca Whirlpool',
 'Lifinity Swap V2',
 

In [14]:
list(vybe_addresses_map.values())

['@solkcrow',
 'Openbook CATO-USDC',
 'Raydium CWAR-USDC',
 'Orca SOL-FISHY',
 'Raydium SBONK-SOL',
 'Magic Eden V2 Authority',
 'Orca SOL-J-SOL',
 'Meteora DLMM HNT-USDC',
 'Raydium mSOL-whETH',
 'verybrickedup',
 'Raydium KIN-USDC',
 'Orca SOL-USDC',
 'Raydium SOL-ZBC',
 'Orca PORT-BONK',
 'Raydium KING-USDC',
 'Orca SOL-DUST',
 'Raydium mSOL-USDT',
 'Orca SOL-PHY',
 'Openbook KART-SOL',
 'Orca SOL-ATLAS',
 'Orca GUAC-USDC',
 'Orca USDC-UPS',
 'Orca UXD-NANA',
 'Openbook MeMe-USDC',
 'Openbook BULL-SOL',
 'Openbook LMDA-SOL',
 'Raydium RBL-USDC',
 'Raydium BBY-USDC',
 'Openbook FRONK-HADES',
 'MarginFi Bonk Asset Pool',
 'Openbook WOOF-USDC',
 'Raydium SOL-SLO',
 'Orca USDC-ISC',
 'Openbook MOLA-USDC',
 'Raydium USDC-JAM',
 'Orca ORCA-USDC',
 'Raydium SOL-WSSI',
 'Openbook AFSeX-SOL',
 'Orca ORCA-stSOL',
 'Raydium LAMDA-USDC',
 'Raydium SOL-BLAKE',
 'Orca SOL-stSOL',
 'Raydium MILK-USDC',
 'Coinbase Exchange 1',
 'Raydium SOL-RAY',
 'Raydium 7SBw57eejb7gob2GM7EWh1xqxS1hBvwWqRmg8aLjWo

# Helius web3 Names

I think this endpoint is deprecated

In [15]:
from helius import NameAPI
HELIUS_API_KEY = os.getenv('HELIUS_API_KEY')

name_api  = NameAPI(HELIUS_API_KEY)


In [16]:
# name = name_api.get_address_name(test_address)
# print(name)

# Metasleuth Labels

In [17]:
METASLEUTH_API_KEY = os.getenv('METASLEUTH_API_KEY')

In [18]:
# import requests

# response = requests.get(
#     "https://aml.blocksec.com/address-label/api/v3/chain-list",
#     headers={"API-KEY":METASLEUTH_API_KEY},
# )

# data = response.json()

In [19]:
# print(data['data'])

In [20]:
def get_solana_chainid(data):
    for d in data['data']:
        chain_id = d.get('chain_id')
        chain_name = d.get('chain_name')

        if chain_name.lower() == 'solana':
            return {chain_id:chain_name}

In [21]:
# solana_metadata = get_solana_chainid(data)


In [22]:
# list(solana_metadata.keys())[0]

In [23]:
filtered_addresses = unique_addresses[:100] # current API key only allows 100 addresses per day, this way we get latest 100
len(filtered_addresses)

100

In [24]:
def chunked(iterable, size):
    """Yield successive `size`-sized chunks from iterable."""
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

def fetch_address_labels(unique_addresses, chain_id, api_key, delay=0.2):
    url = "https://aml.blocksec.com/address-label/api/v3/batch-labels"
    headers = {
        "API-KEY": api_key,
        "Content-Type": "application/json"
    }
    results = []

    for chunk in chunked(unique_addresses, 100):
        response = requests.post(
            url,
            headers=headers,
            json={
                "chain_id": chain_id,
                "addresses": chunk
            }
        )
        data = response.json()
        print(f'data: {data}')
        if data.get("code") == 200000:
            results.extend(data.get("data", []))
        else:
            print(f"[Error] Batch failed: {data}")
        time.sleep(delay)  # optional delay to avoid rate limits

    return results

In [25]:
filtered_addresses

['HUy6XZSKzV7Re64iwnkMpWKSY8chZLF33vhrGdptTUHc',
 'DkNrWSTiGoqU2JigKcLu4J7wVhw18AhZ2MVCVaSdHNtS',
 '9bFNrXNb2WTx8fMHXCheaZqkLZ3YCCaiqTftHxeintHy',
 'EiMGPT4XLRMywc4NUNNjakuMbwyjEr11S2fwU8uPeFvs',
 '9jaHQfFgxqFPDv57BRwR24XCCqq3jNp72bGE9597kRFZ',
 'H1HQpTpWj7YqM2WW5PggzrZKWs2kyi6Be36WHLdUUTKP',
 '9tXiuRRw7kbejLhZXtxDxYs2REe43uH2e7k1kocgdM9B',
 '8bEEDJksHikUUoYNzoax1EGmtHytNSC7ZnNLQKeE933',
 'E815CQsE3j6cnPinfGPaNk9oPKyafBsvSXkXoeUyssGM',
 'rAH5n5D6FYY9gExc9VybLJWKveXiQaDiXT1Yb95k7Zc',
 '2LGJmo5ZoZXRZp5LSzLhfn3GwQzgLvNd5uarL8XrRiYq',
 '6qCVc9PbXnpfr1w2ub6g3AiNUq8QYnckSkoGwQ89xPjB',
 'CapuXNQoDviLvU1PxFiizLgPNQCxrsag1uMeyk6zLVps',
 '8YAQXVFSxFdwHKh9PMnPJtKSU24xLCbpGyU1fsToLotv',
 '6UgHXuw8v4f1X2EC7WQuTd8JcTzLmoMnPujT27nUsNZm',
 'Efnr2xpnC5nMsxpX3NtqWvDzrPBYp6wVAJUxUf4kv9g3',
 '6uMeAJJeWmFYo8mdxscJyE9z8kkDboLXYnJtoXprHWRN',
 'F5QSMAVWXak2YX5LFCcEdnzdxRnx9W64dcLrezkTtqiT',
 '5vzVKMRsVWiaM63ELgQbm96aYY3FVakb3pXoAuRqQ7Wi',
 '8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf',
 'F1ygSdabTQFBKJgMPSxN

In [26]:
# solana_labels = fetch_address_labels(
#     unique_addresses=filtered_addresses,
#     chain_id=-3,  # SOLANA
#     api_key=METASLEUTH_API_KEY
# )


In [27]:
# with open('../data/raw/metasleuth_labels.json', 'w') as f:
#             json.dump(solana_labels, f)

In [28]:
with open('../data/raw/metasleuth_labels.json') as f:
    solana_labels = json.load(f)

In [29]:
len(solana_labels)

16

In [30]:
solana_labels[0].keys()

dict_keys(['chain_id', 'address', 'main_entity', 'main_entity_info', 'comp_entities', 'attributes', 'name_tag'])

In [31]:
entities_dict = {}

for l in solana_labels:
    entity = l['main_entity']
    address = l['address']

    entities_dict[address] = entity

In [32]:
list(entities_dict.values())

['Orca',
 'Coinbase Exchange',
 'MEXC',
 'Jito',
 'Jupiter',
 'Jupiter',
 'Jupiter',
 'Orca',
 'Orca',
 'Orca',
 'Jupiter',
 'Phantom',
 'Jupiter',
 'Jupiter',
 'Coinbase Exchange',
 'Jupiter']

# SolanaScan

In [33]:
SOLSCAN_API_KEY = os.getenv('SOLSCAN_API_KEY')

In [34]:
# url = "https://pro-api.solscan.io/v2.0/transaction/detail?tx=65mbqVyFo9kGav1zdCTUjdYtVZkRb82m2E9ZqedDwWdKDiPKuxNKmMsVHFWnBxMin2SncasNB8FGmMM7ANRFRtLn"    

# headers = {"token":SOLSCAN_API_KEY}

# response = requests.get(url, headers=headers)

# print(response.text)

# data = response.json()


In [35]:
# data['data'].keys()

# Combining all Label Maps to DF

In [36]:
entities_dict

{'5snaYowgJDfuM1LPbTNUYHbgkKHtVVnzHiiLDWUV2hh8': 'Orca',
 '2AQdpHJ2JpcEgPiATUXjQxA8QmafFegfQwSLWSprPicm': 'Coinbase Exchange',
 'ASTyfSima4LLAdDgoFGkgqoKowG1LZFDr9fAQrg7iaJZ': 'MEXC',
 'ADuUkR4vqLUMWXxW9gh6D6L8pMSawimctcNZ5pGwDcEt': 'Jito',
 'CapuXNQoDviLvU1PxFiizLgPNQCxrsag1uMeyk6zLVps': 'Jupiter',
 '45ruCyfdRkWpRNGEqWzjCiXRHkZs8WXCLQ67Pnpye7Hp': 'Jupiter',
 '4xDsmeTWPNjgSVSS1VTfzFq3iHZhp77ffPkAmkZkdu71': 'Jupiter',
 'GUWM1arUyDnkMGCHvJu3yt1qomJ988utqC3dFN2AUCDT': 'Orca',
 '7qbRF6YsyGuLUVs6Y1q64bdVrfe4ZcUUz1JRdoVNUJnm': 'Orca',
 'HQcY5n2zP6rW74fyFEhWeBd3LnJpBcZechkvJpmdb8cx': 'Orca',
 '2MFoS3MPtvyQ4Wh4M9pdfPjz6UhVoNbFbGJAskCPCj3h': 'Jupiter',
 'tzvXws1qhmfdPkPcprezULCDQPAJqzPhbZ3SMrqRPNE': 'Phantom',
 '9nnLbotNTcUhvbrsA6Mdkx45Sm82G35zo28AqUvjExn8': 'Jupiter',
 '6LXutJvKUw8Q5ue2gCgKHQdAN4suWW8awzFVC6XCguFx': 'Jupiter',
 'H8sMJSCQxfKiFTCfDR3DUMLPwcRbM61LGFJ8N4dK3WjS': 'Coinbase Exchange',
 'BQ72nSv9f3PRyRKCBnHLVrerrv37CYTHm5h3s9VSGQDV': 'Jupiter'}

In [37]:
vybe_programs_map

{'sageqqfewepdhh6hmdcmwy7yjhppykldnrxkb3ki8e6': 'Star Atlas SAGE',
 'mgr99qfmybytqgpwmnqunv7vblmwwxdsrhufv8jf3jm': 'Cardinal Token Manager',
 'pyts9tjg1qyazypk7n8rw8gfw9suaqqyymhjq4e7jcq': 'Pyth Staking',
 'pfacsruobdmvfmkq1bazwj27t6d2gjhschb1vcfnrmq': 'Star Atlas - PROFILE ACTION',
 'hadek9dlv9ea7ya5kctqsvsvrzejc3jgd5a9y3cnbvu': 'Hadeswap',
 'hvsrnc3nkbcryqds2docyhz9ypkevzdsjqg6rvtk1s8': 'Helium Voter Stake Registry',
 '92wdgeqyidkrcbfhobtg8hxmj932xwerckacigsw3umr': 'HXRO - Risk',
 'jcfrapv7852esrwjjgry2mysumydxzgvvhrmlmexvmvp': 'Citrus',
 'hauss13jsjafwwwgqztuqrmwyvyxn9eqpqmwv1pbbmk': 'MPL Auction House',
 'zetaxsqbrek56dhigxrn75yj2nhu3ayunxvhxpkf3ad': 'Zeta Markets',
 'craftf1egzeopfj1rpatsqg1f6hhrrbaf4gro9hdszjr': 'Star Atlas - CRAFTING',
 'mfv2hwf31z9kbca1snepyctwafyhdvnv7fznsebvaca': 'Marginfi',
 '6f3n8a6fccrb8hxnpv3t2ludfmjk6vby7miqcjpuuo6a': 'Orca-Openbook Bot',
 'swapnyd8xiqwj6ianp9snpu4bruqfxadzvhebnaxjjz': 'Stabble Stable Swap',
 'monacouxktui6vksqwalyxmxksievfnwecyxtgkbcih'

In [38]:
# Normalize mapping keys
vybe_programs_map = {k.lower(): v for k, v in vybe_programs_map.items()}
vybe_addresses_map = {k.lower(): v for k, v in vybe_addresses_map.items()}
entities_dict = {k.lower(): v for k, v in entities_dict.items()} # metasleuth
flipside_labels_dict = {k.lower(): v for k, v in flipside_labels_dict.items()} # flipside

In [39]:
combined_address_label_map = {
    **vybe_addresses_map,
    **entities_dict,
    **flipside_labels_dict
}
len(combined_address_label_map.keys())

105967

In [40]:


# Apply human-readable labels to tx-level DataFrame
test_wallet_timeseries['sender_name'] = test_wallet_timeseries['sender'].astype(str).str.lower().map(
    lambda addr: combined_address_label_map.get(addr, 'Unknown Address'))

test_wallet_timeseries['receiver_name'] = test_wallet_timeseries['receiver'].astype(str).str.lower().map(
    lambda addr: combined_address_label_map.get(addr, 'Unknown Address'))

test_wallet_timeseries['counterparty_name'] = test_wallet_timeseries['counterparty'].astype(str).str.lower().map(
    lambda addr: combined_address_label_map.get(addr, 'Unknown Address'))

test_wallet_timeseries['program_name'] = test_wallet_timeseries['program_id'].astype(str).str.lower().map(
    lambda pid: vybe_programs_map.get(pid, 'Unknown Program'))



In [41]:
test_wallet_timeseries['wallet_entity_label'] = combined_address_label_map.get(test_address.lower(), 'Unknown Entity')

In [42]:
test_wallet_timeseries[test_wallet_timeseries['receiver_name']!='Unknown Address']['receiver_name'].unique()

array(['Orca', 'Jupiter', 'Jupiter Aggregator Authority',
       'tip payment account', 'Orca SOL-USDC', 'Phantom', 'Orca SOL-bSOL',
       'kamino fee wallet', 'Kamino Reserve 1', 'Orca SOL-stSOL',
       'Raydium AMM Authority'], dtype=object)

In [43]:
test_wallet_timeseries[test_wallet_timeseries['sender_name']!='Unknown Address']['sender_name'].unique()

array(['Orca', 'Jupiter', 'MEXC', 'Jupiter Aggregator Authority',
       'Orca SOL-USDC', 'Coinbase Exchange', 'Orca SOL-bSOL',
       'Kamino Reserve 1', 'SolBlaze Airdrop Wallet',
       'coinbase hot wallet', 'Orca SOL-stSOL', 'Raydium AMM Authority'],
      dtype=object)

In [44]:
test_wallet_timeseries[test_wallet_timeseries['program_name']!='Unknown Program']['program_name'].unique()

array(['Jupiter Swap V6', 'Allbridge Core', 'MPL Bubblegum',
       'PENGU Airdrop Claiming', 'Jupiter Swap V2', 'Parcl V3',
       'Kamino Program', 'Kamino Lend'], dtype=object)

In [45]:
test_wallet_timeseries.to_csv('../data/processed/test_acc_timeseries_labeled.csv', index=False)